In [9]:
P_generated = 2.95 # W
P_base = 1.518 # W
E_pre = 0.01 # J
R_frame = 2.5 # Hz
R_cloudy = 0.444
R_sunny = 0.408
R_filter = 1 - (R_sunny * (1 - R_cloudy))
bitrate = 80_000_000 # bps
comm_power = 20 # W
E_sendbit = 1/bitrate * comm_power # J/bit
contact_r = 0.0075
B_downlink = bitrate * contact_r # bps
T_pre = 0.038 # s
S_frame = 256*256*13*8 # bits

In [10]:
print(f"E_sendbit: {E_sendbit} J/bit")
print(f"B_downlink: {B_downlink} bps")
print(f"S_frame: {S_frame} bits")
print(f"R_filter: {R_filter}")

E_sendbit: 2.5e-07 J/bit
B_downlink: 600000.0 bps
S_frame: 6815744 bits
R_filter: 0.7731520000000001


In [11]:
fn = ["methane", "segment", "vessel", "wildfire", "moisture"]

Ef = {
    "methane": 0.041,
    "segment": 0.755,
    "vessel": 1.053,
    "wildfire": 0.667,
    "moisture": 0.092,
} # J

Cf = {
    "methane": 0.051,
    "segment": 0.047,
    "vessel": 0.026,
    "wildfire": 0.028,
    "moisture": 0.041,
}

Tf = {
    "methane": 0.018,
    "segment": 0.494,
    "vessel": 0.585,
    "wildfire": 0.353,
    "moisture": 0.050,
} # s

In [12]:
def analyze_constraints(selected_fns):
    P_compute = P_base + E_pre * R_frame + sum([Ef[fn] * R_frame * (1-R_filter) for fn in selected_fns])
    P_comm = E_sendbit * sum([Cf[fn] * R_frame * (1-R_filter) * S_frame for fn in selected_fns])

    print(f"P_generated >= P_compute + P_comm: {P_generated >= P_compute + P_comm}")
    print(f"P_compute: {P_compute}")
    print(f"P_comm: {P_comm}")
    print(f"P_generated: {P_generated}")

    downlink = sum([Cf[fn] * R_frame * (1-R_filter) * S_frame for fn in selected_fns])

    print(f"B_downlink >= downlink: {B_downlink >= downlink}")
    print(f"Downlink: {downlink} bps")
    print(f"B_downlink: {B_downlink} bps")

    processing_time = T_pre + sum([Tf[fn] * (1-R_filter) for fn in selected_fns])

    print(f"R_frame^-1 >= processing_time: {R_frame**-1 >= processing_time}")
    print(f"Processing time: {processing_time}")
    print(f"R_frame^-1: {R_frame**-1}")

In [13]:
selected_fns = ["methane", "segment", "vessel", "wildfire", "moisture"]

analyze_constraints(selected_fns)

P_generated >= P_compute + P_comm: False
P_compute: 3.0220489599999993
P_comm: 0.18650288357375994
P_generated: 2.95
B_downlink >= downlink: False
Downlink: 746011.5342950398 bps
B_downlink: 600000.0 bps
R_frame^-1 >= processing_time: True
Processing time: 0.3782719999999999
R_frame^-1: 0.4


In [14]:
selected_fns = ["methane", "vessel", "wildfire", "moisture"]
analyze_constraints(selected_fns)

P_generated >= P_compute + P_comm: True
P_compute: 2.5938733599999995
P_comm: 0.14108508291071997
P_generated: 2.95
B_downlink >= downlink: True
Downlink: 564340.3316428799 bps
B_downlink: 600000.0 bps
R_frame^-1 >= processing_time: True
Processing time: 0.2662090879999999
R_frame^-1: 0.4
